In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from time import time
import datetime
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import ComplementNB
from sklearn import neighbors
from xgboost import XGBClassifier as XGBC
from sklearn.neural_network import MLPClassifier
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [ ]:
# Import and definition
data = pd.read_csv(r"D:\jiqixuexilianxishuju\jieguo.csv",index_col = 0) 
X = data.iloc[:,1:]
Y= data.iloc[:,0]
# Delete blank value
X.isnull().sum()
X.dropna(axis=1,inplace=True)
X1 = X.values
Y1 = Y.values
# Feature selection
feat_labels = X.columns.values.tolist()
forest = RandomForestClassifier(random_state = 10)
forest.fit(X1, Y)
for f in range(X1.shape[1]):   # x_train.shape[1]=13
    print("%2d) %-*s %f" % \
          (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))
threshold = 0.004
X2 = X1[:, importances >= threshold]
# Divide training set and test set
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X2,Y,test_size=0.3,random_state=0)

In [ ]:
# RF model
superpa1 = []
superpa2 = []
for i in range(1,100001,10000):
    rfc = RandomForestClassifier(n_estimators=i, random_state=0, n_jobs=-1,max_depth=4,class_weight='balanced')
    rfc = rfc.fit(Xtrain, Ytrain)
    superpa1.append(accuracy_score(rfc.predict(Xtrain),Ytrain))
    superpa2.append(accuracy_score(rfc.predict(Xtest),Ytest))
print(max(superpa2),superpa2.index(max(superpa2))) # 取出结果中的最大值
graph = [superpa1,superpa2]
color = ["black","gray"]
label = ["RF","RFtest"]
plt.figure(figsize=(40,20))
for i in range(len(graph)):
    plt.plot(np.arange(1,100001,10000),graph[i],color[i],label=label[i])
plt.legend(loc=4)
plt.xticks(np.arange(1,100001,10000))
plt.show()

times = time()
RF = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1,max_depth=4,class_weight='balanced')
RF.fit(Xtrain, Ytrain)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

result_1 =RF.predict(Xtrain)
result =RF.predict(Xtest)
# Accuracy
score = RF.score(Xtrain,Ytrain)
score = RF.score(Xtest, Ytest)
# Recall
metrics.recall_score(Ytrain, result_1, average='weighted')
metrics.recall_score(Ytest, result, average='weighted')
# F1 Score
metrics.f1_score(Ytrain, result_1, average='weighted')
metrics.f1_score(Ytest, result, average='weighted')  
# Confusion matrix
confusion_matrix(Ytrain, result_1)
confusion_matrix(Ytest, result)
# AUC
ypred = RF.predict_proba(Xtrain)
roc_auc_score(Ytrain,ypred,multi_class='ovo')
ypred = RF.predict_proba(Xtest)
roc_auc_score(Ytest,ypred,multi_class='ovo')
# precision
precision_score(Ytrain, result_1, average='weighted')
precision_score(Ytest, result, average='weighted')

In [ ]:
# LR model
from sklearn.metrics import accuracy_score
l2 = []
l2test = []
for i in np.arange(2001,2201,10):
    lrl2 = LR(multi_class='multinomial',solver='lbfgs',max_iter=i)
    lrl2 = lrl2.fit(Xtrain,Ytrain)
    l2.append(accuracy_score(lrl2.predict(Xtrain),Ytrain))
    l2test.append(accuracy_score(lrl2.predict(Xtest),Ytest))
graph = [l2,l2test]
color = ["black","gray"]
label = ["L2","L2test"]
plt.figure(figsize=(20,5))
for i in range(len(graph)):
    plt.plot(np.arange(2001,2201,10),graph[i],color[i],label=label[i])
plt.legend(loc=4)
plt.xticks(np.arange(2001,2201,10))
plt.show()

times = time()
lr = LR(multi_class='multinomial',solver='lbfgs',max_iter=2041)
lr.fit(Xtrain,Ytrain)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))
# The evaluation parameters of the model refer to the random forest model

In [ ]:
# SVC model
Kernel = ["linear","poly","rbf","sigmoid"]
for kernel in Kernel:    
    clf= SVC(kernel = kernel
             , gamma="auto"
             , degree = 1
             , cache_size=5000
             ,max_iter=10000
             ,class_weight = 'balanced'
             ,probability=True
            ).fit(Xtrain,Ytrain)
    print("The accuracy under kernel %s is %f" % (kernel,clf.score(Xtest,Ytest)))
    
times = time()
clf = SVC(kernel = "rbf"
              ,gamma="scale"
              ,C=1000
              ,cache_size = 6000
              ,coef0 = 0
              ,random_state = 0
              # ,class_weight = 'balanced'
              ,probability=True
             ).fit(Xtrain, Ytrain) 
result2 = clf.predict(Xtest)
score = clf.score(Xtest,Ytest)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

In [ ]:
# Bayesian model needs to be normalized
mms = MinMaxScaler().fit(Xtrain)
Xtrain_2 = mms.transform(Xtrain)
Xtest_2 = mms.transform(Xtest)
# CNB model
times = time()
cnb = ComplementNB(alpha =1.3).fit(Xtrain_2,Ytrain)
result5 = cnb.predict(Xtest_2)
acc_score = cnb.score(Xtest_2,Ytest)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

In [ ]:
# KNN model
for n_neighbors in range(1,50,1):
    for weights in ['uniform', 'distance']:
        nei = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        nei.fit(Xtrain, Ytrain)
        nei.score(Xtest,Ytest)
        print('模型得分', nei.score(Xtest,Ytest))

times = time()
nei = neighbors.KNeighborsClassifier(n_neighbors = 40, weights='uniform')
nei.fit(Xtrain, Ytrain)
nei.score(Xtest,Ytest)
result6 = nei.predict(Xtest)
print('模型得分', nei.score(Xtest,Ytest))
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

In [ ]:
# XGBoost model
times = time()
from xgboost import XGBClassifier as XGBC
sxgb = XGBC(n_estimators = 500
            ,booster='gbtree'
            ,objective = 'multi:softmax'
            ,gamma = 3
            ,max_depth = 3
            ,reg_lambda = 3
            ,subsample = 0.9
            ,colsample_bytree =0.8
            ,min_child_weight = 1
            ,eta = 0.3
            ,random_state =0).fit(Xtrain,Ytrain)
ypred = sxgb.predict(Xtest)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

# AUC needs to predict the probability value
sxgb2 = XGBC(n_estimators = 500
            ,booster='gbtree'
            ,objective = 'multi:softprob'
            ,gamma = 3
            ,max_depth = 3
            ,reg_lambda = 3
            ,subsample = 0.9
            ,colsample_bytree =0.8
            ,min_child_weight = 1
            ,eta = 0.3
            ,random_state =0).fit(Xtrain,Ytrain)
ypred2 = sxgb2.predict_proba(Xtest)
ypred3 = sxgb2.predict_proba(Xtrain)

In [ ]:
# MLP model
times = time()
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 5), learning_rate='constant',
       learning_rate_init=0.0001, max_iter=10000, momentum=0.7,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.3, verbose=False,
       warm_start=False).fit(Xtrain,Ytrain)
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))